# **House Prices Predication**
 * Model Selection 
 * Classification regressor 
 * Ensemble Learning

# **Importing**

In [ ]:
import numpy as np 
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score , mean_absolute_percentage_error
from matplotlib import pyplot as plt 
import seaborn as sns 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor 
from xgboost import XGBRegressor 
import lightgbm as ltb
from sklearn.ensemble import StackingRegressor


from sklearn.metrics import precision_recall_fscore_support 
from sklearn.metrics import accuracy_score,classification_report,precision_recall_fscore_support
from sklearn.preprocessing import label_binarize
 
from sklearn.naive_bayes import CategoricalNB, GaussianNB, ComplementNB, BernoulliNB, MultinomialNB 
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score , mean_absolute_percentage_error

# **Reading Data**

In [ ]:
dataset_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
dataset_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
dataset_test_ids = dataset_test['Id']

# Snap of data
dataset_train.head()


In [ ]:
dataset_train.describe()

# **Pre-processing The Dataset**

In [ ]:
# Checking Empty cells in Trainning data

pd.set_option('display.max_rows',None) # for showing all columns

dataset_train.isna().sum()

In [ ]:
# Checking Empty cells in Trainning data

pd.set_option('display.max_rows',None) # for showing all columns

dataset_test.isna().sum()

In [ ]:
 # Handle Data issues 
    


 # Drop cols
dataset_train.drop(columns= ["Fence",'PoolQC','MiscFeature','Alley' ,'Id'] , axis = 1 , inplace=True )
dataset_test.drop(columns= ["Fence",'PoolQC','MiscFeature','Alley'  , 'Id'] , axis = 1 , inplace=True )

# Fill with mean
        
dataset_train["LotFrontage"].fillna(dataset_train["LotFrontage"].mean() ,inplace=True )
dataset_test["LotFrontage"].fillna(dataset_train["LotFrontage"].mean() ,inplace=True )


# Fill with Mode 
dataset_train["FireplaceQu"].fillna(dataset_train["FireplaceQu"].mode()[0] ,inplace=True )
dataset_test["FireplaceQu"].fillna(dataset_train["FireplaceQu"].mode()[0] ,inplace=True )


# drop small entries 

dataset_train.dropna(inplace=True )

# Filling Remaining Empty cells in testing dataset with mean per column 
for i in dataset_test.columns:
    if( dataset_test[i].isna().values.any()):
        dataset_test[i].fillna(dataset_test[i].mode()[0] , inplace=True  )




dataset_train.isna().sum()



In [ ]:
dataset_test.isna().sum()


In [ ]:
dataset_train.head()

In [ ]:
dataset_test.head()

In [ ]:
# Labeling Categrocal Data 

# Getting Categrocal cols 
categorical_cols = dataset_train.select_dtypes(include=['object']).columns.tolist()


for nc in categorical_cols:
 dataset_train[nc]= LabelEncoder().fit_transform(dataset_train[nc],)
 dataset_test[nc]= LabelEncoder().fit_transform(dataset_test[nc],)



In [ ]:
# Splitting Features from Output in Dataset
features_cols_tarin = dataset_train.drop(columns = ['SalePrice' ],axis=1)
output_cols_train =  dataset_train['SalePrice']


# Splitting Trainning data  for evalutaing 
features_train, features_test, output_train, output_test = train_test_split (features_cols_tarin, output_cols_train,
   test_size = 0.30,      random_state = 11,)



In [ ]:
# Normalizing Data ranges 

normalizing_scaller =MinMaxScaler()  
normalizing_scaller.fit(features_train)
features_cols_train_normlized = normalizing_scaller.transform(features_train) 
features_cols_test_normlized = normalizing_scaller.transform(features_test) 

# normalize test data
features_cols_real_test = normalizing_scaller.transform(dataset_test  )



print(f"Normlized Features its max : \n{features_cols_train_normlized.max()} \n\nits min : \n{features_cols_train_normlized.min()}" )

#            x                  y
# train -> features_cols_train_normlized , output_train
# test -> features_cols_test_normlized , output_test


# **Model Selection**

Linear Regression

In [ ]:
# Fitting Linear Regression to the dataset

# Train
regressor = LinearRegression() 
regressor.fit(features_cols_train_normlized, output_train)
# Predict
output_predicted = regressor.predict(features_cols_test_normlized)

# Metric
MAPE =	MAPE =	mean_absolute_percentage_error(output_test, output_predicted)

print(f"MAPE for Linear Regression : {round(MAPE*100,2)}%")

Polynomial Regression

In [ ]:
# Fitting Polynomial Regression to the dataset
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree=3)
X_poly = poly_reg.fit_transform(features_cols_train_normlized)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, output_train)

# Predict
output_predicted2=pol_reg.predict(poly_reg.transform(features_cols_test_normlized))

MAPE_Poly=mean_absolute_percentage_error(output_test, output_predicted2)
print(f"MAPE for Polynomial Regression : {round(MAPE_Poly*100,2)}%")

Classifcation Regressors 

In [ ]:
# KNN 
regressor_knn = KNeighborsRegressor(n_neighbors=4) 
regressor_knn.fit(features_cols_train_normlized, output_train)
y_pred_knn = regressor_knn.predict(features_cols_test_normlized)
MAPE_knn	= mean_absolute_percentage_error(output_test, y_pred_knn)
print(f'KNN Regressor with 16 Nieghbors , MAPE: {round(MAPE_knn*100,2)}%')

# SVR 
regressor_svr = SVR(kernel='rbf') 
regressor_svr.fit(features_cols_train_normlized, output_train)
y_pred_svr = regressor_svr.predict(features_cols_test_normlized)
MAPE_svr = mean_absolute_percentage_error(output_test, y_pred_svr)
print(f'SVR Regressor For kernel rbf , MAPE: {round(MAPE_svr*100,2)}%')

# Decision Tree Regressor
regressor_DT = DecisionTreeRegressor() 
regressor_DT.fit(features_cols_train_normlized, output_train)
y_pred_DT = regressor_DT.predict(features_cols_test_normlized)
MAPE_dt	= mean_absolute_percentage_error(output_test, y_pred_DT)
print(f'Decison Tree Regressor ,  MAPE: {round(MAPE_dt*100,2)}%')


Ensemble Learning Models

In [ ]:
## Ensemble Learning
max_estimators= 200

# Random Forest Tree 
regressor_RF= 	RandomForestRegressor(n_estimators=max_estimators)
regressor_RF.fit(features_cols_train_normlized, output_train)

# Gradient Boosting  
regressor_GB = GradientBoostingRegressor(n_estimators=max_estimators) 
regressor_GB.fit(features_cols_train_normlized, output_train)


# XGB
regressor_XGB	= XGBRegressor(n_estimators=max_estimators) 
regressor_XGB.fit(features_cols_train_normlized, output_train)

# LGb 
regressor_LGBM =	ltb.LGBMRegressor(n_estimators=max_estimators) 
regressor_LGBM.fit(features_cols_train_normlized, output_train)

 
# There Metric
y_pred_rf = regressor_XGB.predict(features_cols_test_normlized)
MAPE_rf=	mean_absolute_percentage_error(output_test, y_pred_rf) 
print(f'\nFor Random Forest , MAPE: {round(MAPE_rf*100,2)}%')

y_pred_gb = regressor_GB.predict(features_cols_test_normlized)
MAPE_gb=	mean_absolute_percentage_error(output_test, y_pred_gb) 
print(f'For Gradient Boosting , MAPE: {round(MAPE_gb*100,2)}%')

y_pred_xgb = regressor_XGB.predict(features_cols_test_normlized)
MAPE_xgb=	mean_absolute_percentage_error(output_test, y_pred_xgb) 
print(f'For XGB , MAPE: {round(MAPE_xgb*100,2)}%')

y_pred_lgb = regressor_LGBM.predict(features_cols_test_normlized)
MAPE_lgb=	mean_absolute_percentage_error(output_test, y_pred_lgb) 
print(f'For LGBM , MAPE: {round(MAPE_lgb*100,2)}%')

In [ ]:
# Stacking Trees 
estimators = [
     ('knn', KNeighborsRegressor(n_neighbors=5) ),
     ('svr',  SVR(kernel='rbf') )]
regressor_stacking = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())
regressor_stacking.fit(features_cols_train_normlized, output_train)

y_pred_stacking = regressor_stacking.predict(features_cols_test_normlized)
MAPE_stacking=	mean_absolute_percentage_error(output_test, y_pred_stacking) 
print(f'For Stacking Ensemble , MAPE: {round(MAPE_stacking*100,2)}%')


# **Predicting with Testing Data with Best Model**

As we've got our best model is Gradient Boosting Regressor

In [ ]:
# Gradient Bossting Model 
predictions_gb = regressor_GB.predict (features_cols_real_test)


In [ ]:
# Load the predicted data in csv file 

submission = pd.DataFrame({'Id':dataset_test_ids, 'SalePrice':predictions_gb})
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()